# Import Modules

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from scipy.fftpack import dct, idct
import heapq

# Implementation

Quantization Matrix

In [ ]:
quantization = np.array([
    [16, 11, 10, 16, 24, 40, 51, 61],
    [12, 12, 14, 19, 26, 58, 60, 55],
    [14, 13, 16, 24, 40, 57, 69, 56],
    [14, 17, 22, 29, 51, 87, 80, 62],
    [18, 22, 37, 56, 68, 109, 103, 77],
    [24, 35, 55, 64, 81, 104, 113, 92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103, 99],
])

quantization_B = np.array([
    [8, 6, 5, 8, 12, 20, 26, 31],
    [6, 6, 7, 10, 13, 29, 30, 28],
    [7, 7, 8, 12, 20, 29, 35, 28],
    [7, 9, 11, 15, 26, 44, 40, 31],
    [9, 11, 19, 28, 34, 55, 52, 39],
    [12, 18, 28, 32, 41, 52, 57, 46],
    [25, 32, 39, 44, 52, 61, 60, 51],
    [36, 46, 48, 49, 56, 50, 52, 50],
])


## Compress

### DCT

In [ ]:
'''
DCT Function
Description:
    Calculate the transformed of a image, breaking it in bloks of 8x8
    Use two times the dct for 1D, each time for one axis of the image
Input: Original image
Output: image transformed
Warning: Need to deal with images that are not multiple of 8
'''

def DCT(img):
    Y = np.zeros_like(img)

    nLines, nCols = img.shape
    U = nLines//8
    V = nCols//8

    for u in range(U):
        for v in range(V):
            # img_blck = np.float32(imf[u*8: u*8 + 8, v*8: v*8 + 8]) # float conversion
            img_blck = img[u*8: u*8 + 8, v*8: v*8 + 8]
            Y[u*8: u*8 + 8, v*8: v*8 + 8] = cv.dct(img_blck)
    
    return Y

### DCT + Quantization

In [ ]:
'''
Function: quant_DCT
Description:
    For each block 8x8 from the image transformed
    Is divided by the quantization matrix

Input: image transformed
Output: image tranformed divided by quantization 
'''

def quant_DCT(Y):
    Y_quant = np.zeros(Y.shape)
    U = Y.shape[0]//8
    V = Y.shape[1]//8

    for u in range(U):
        for v in range(V):
            Y_quant[u*8 : u*8 + 8, v*8 : v*8 + 8] = (Y[u*8 : u*8 + 8, v*8 : v*8 + 8] / quantization_B)
    
    return Y_quant

## Decompress

In [ ]:
def dequant_DCT(Y_quant):
    U = Y_quant.shape[0]//8
    V = Y_quant.shape[1]//8
    Y_dequant = np.zeros(Y_quant.shape)
    
    for u in range(U):
        for v in range(V):
            Y_dequant[u*8 : u*8 + 8, v*8 : v*8 + 8] = Y_quant[u*8 : u*8 + 8, v*8 : v*8 + 8] * quantization_B
    
    return Y_dequant



In [ ]:
def iDCT(Y):

    U = Y.shape[0]//8
    V = Y.shape[1]//8
    img_rec = np.zeros(Y.shape)
    for u in range(U):
        for v in range(V):
            img_rec[u*8 : u*8 + 8, v*8 : v*8 + 8] = cv.idct(Y[u*8 : u*8 + 8, v*8 : v*8 + 8])
    
    return img_rec

# Execution and Test

## Load Image

In [ ]:
img = cv.imread('imgs/listrada.png', cv.IMREAD_GRAYSCALE)

plt.imshow(img, cmap='gray')

## Compress


### DCT

In [ ]:
imf = np.float32(img) - 128
Y = DCT(imf)

In [ ]:
plt.imshow((Y), cmap='gray')
(np.unique(Y).size, np.unique(img).size)

### Quantizando

In [ ]:
Y_quant = quant_DCT(Y)
# Y_quant = np.round(quant_DCT(Y))
Y_quant = np.trunc(quant_DCT(Y))
# plt.hist(Y_quant)
x = 0

In [ ]:

nums = np.unique(Y_quant)
print(f'Imagem Original: {np.unique(img).size}    |     Imagem Tranformada Quantizada: {np.unique(Y_quant).size}')
print(f'Numero de Zeros na Imagem {(Y_quant == 0).sum()},  Numero de Pixels da Imagem {img.size}')

### Huffman

In [ ]:
'''
Func: weight_grey_tons
Description:
    Get the number of occurrence of each color
Input: quantizated transformed image (Future: Error quantizated transformed image)
Output: table with the occurrence of each color
    [0     4] -> color 0 -> 4 times
    [1     1] -> color 1 -> 1 time
    [2     5]
    [3     6]
'''

def weight_grey_tons(img):
    # depth = 8
    # nLine, nCols = img.shape
    # hist = np.zeros((2**8))
    
    #get unique values and counts of each value
    unique, counts = np.unique(img.flatten(), return_counts=True)

    #display unique values and counts side by side
    occurrence = np.asarray((unique, counts)).T

    return occurrence

'''
Func: huffman_tree
Description:
    Create the the tree representation for the Huff Algorithm
Input: Color occurency 
    Accept Format
    [0     4] -> color 0 -> 4 times
    [1     1] -> color 1 -> 1 time
    [2     5]
    [3     6]

Output: Tree

To-do
    Priority_queue
    Tree
'''

def huffman_tree(ocurrence):

    nLine, _ = ocurrence.shape
    
    class Tree:
        def __init__(self, data):
            self.left = None
            self.right = None
            self.data = data

    forest = []

    # Populate the list
    for i in range (nLine):
        color, times =  ocurrence[i]
        tree = BinaryTree(color)
        forest.append((times, color, tree)) # Why a tuple? Is easier to use a tuple than define a __lt__ to compare the objects

    # Create a priority list
    heapq.heapify(forest)

    # Variable to Simulate a Stable Sort
    # And to sort first the leaf with data -> (data is a value of the gray scale)
    count = 300
    while(len(forest) > 1):
        f_times, f_color, f_tree = heapq.heappop(forest)
        s_times, s_color, s_tree = heapq.heappop(forest)

        new_tree = BinaryTree(None)
        new_tree.left = f_tree
        new_tree.right = s_tree
        heapq.heappush(forest, (f_times + s_times, count, new_tree))
        count += 1
    

    _, _, h_tree = heapq.heappop(forest)
    
    def printPostorder(node):
 
        if node:
    
            # First recur on left child
            printPostorder(node.left)
    
            # the recur on right child
            printPostorder(node.right)
    
            # now print the data of node
            if node.data != None:
                print(f'1c"{node.data}"')
            else:
                print(0, end="")
    

    printPostorder(h_tree)
    # Mark the end of the tree
    print(0)
    return h_tree

    
    


In [ ]:
class BinaryTree:
    def __init__(self, data):
        self.data = data
        self.right = None
        self.left = None
    def insert(self, data):
        if self.data == data:
            return
        elif self.data < data:
            if self.right is None:
                self.right = BinaryTree(data)
            else:
                self.right.insert(data)
        else: # self.data > data
            if self.left is None:
                self.left = BinaryTree(data)
            else:
                self.left.insert(data)
    def display(self):
        lines, *_ = self._display_aux()
        for line in lines:
            print(line)
    def _display_aux(self):
        """Returns list of strings, width, height, and horizontal coordinate of the root."""
        # No child.
        if self.right is None and self.left is None:
            line = '%s' % self.data
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle
        # Only left child.
        if self.right is None:
            lines, n, p, x = self.left._display_aux()
            s = '%s' % self.data
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s
            second_line = x * ' ' + '/' + (n - x - 1 + u) * ' '
            shifted_lines = [line + u * ' ' for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2
        # Only right child.
        if self.left is None:
            lines, n, p, x = self.right._display_aux()
            s = '%s' % self.data
            u = len(s)
            first_line = s + x * '_' + (n - x) * ' '
            second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
            shifted_lines = [u * ' ' + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2
        # Two children.
        left, n, p, x = self.left._display_aux()
        right, m, q, y = self.right._display_aux()
        s = '%s' % self.data
        u = len(s)
        first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
        second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
        if p < q:
            left += [n * ' '] * (q - p)
        elif q < p:
            right += [m * ' '] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2

#### Teste

In [ ]:
#get unique values and counts of each value
unique, counts = np.unique(img, return_counts=True)

#display unique values and counts side by side
print(np.asarray((unique, counts)).T)

In [ ]:
output = weight_grey_tons(img)
huffman_tree(output).display()

In [ ]:
plt.hist(img, density=True)
x = 0

## Unconpress

### Huffman

### Dequantizando

In [ ]:
Y_dequant = dequant_DCT(Y_quant)
np.unique(Y_dequant).max()

### Transformada inversa

In [ ]:
img_rec = iDCT(Y_dequant)
img_rec += 128

# Corrige os pixies que estouraram
filtro = img_rec > 255
img_rec[filtro] = 255

filtro = img_rec < 0
img_rec[filtro] = 0
# img_rec *= rate
img_rec = np.round(img_rec)

In [ ]:
plt.imshow((img_rec), cmap='gray')

In [ ]:
plt.imshow((img), cmap='gray')

# Análises


In [ ]:
print(f'MSE: {((img - img_rec)**2).sum()/img.size}')
print(f'ME: {np.abs(img - img_rec).max()}')

In [ ]:
# histogramas de Y e Y_quant para, posteriormente, criar matriz de erros para huffman
# observe o eixo-x, i.e. os erros serão mais próximos
plt.hist(Y)
plt.show()
plt.hist(Y_quant)
plt.show()

In [ ]:
(np.unique(Y).size, np.unique(Y_quant).size)